In [1]:
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np
from web3 import Web3
import tqdm
import matplotlib.pyplot as plt
import yfinance as yf

In [2]:
blocks = pd.read_csv("/Users/gosa/Desktop/internship/data/eth_btc/FirstBlockOfTheDay.csv",
                     names = ['date', 'block'])

In [3]:
BTC_ETH = pd.read_excel('/Users/gosa/Desktop/internship/data/eth_btc/v2_btc_eth_events_15999999_clean.xlsx')

In [4]:
def find_number_location(arr, number):
    for i in range(len(arr) - 1):
        if arr[i] <= number <= arr[i + 1]:
            return i

In [5]:
BTC_ETH = BTC_ETH[['Event', 'Block', 'amount0In', 'amount1In','amount0Out', 'amount1Out' ]]
BTC_ETH

,Event,Block,amount0In,amount1In,amount0Out,amount1Out
0,Mint,10091132,10985500,4.300009e+18,0,0.000000e+00
1,Swap,10091412,0,1.000000e+15,2546,0.000000e+00
2,Swap,10091436,0,1.000000e+16,25400,0.000000e+00
3,Mint,10091445,1101,5.000000e+14,0,0.000000e+00
4,Swap,10091498,0,3.176716e+17,749921,0.000000e+00
...,...,...,...,...,...,...
450373,Swap,15997826,784872,0.000000e+00,0,1.075582e+17
450374,Swap,15998651,4658864,0.000000e+00,0,6.382939e+17
450375,Swap,15998750,0,2.129990e+16,154565,0.000000e+00
450376,Swap,15999034,1452079,0.000000e+00,0,1.988932e+17


In [ ]:
dates = np.array([])
for i in range(len(BTC_ETH)):
    num = find_number_location(blocks['block'].values, BTC_ETH['Block'][i])
    if num:
        dates = np.append(dates, blocks['date'][num])
        
    else:
        dates = np.append(dates, np.nan)

In [ ]:
BTC_ETH['date'] = pd.DataFrame(dates)
BTC_ETH['date'] = pd.to_datetime(BTC_ETH['date'])
BTC_ETH

In [ ]:
def get_data(symbol, start = str(int(dt.datetime(2021,9,12).timestamp()*1000)),  
             end = str(int(dt.datetime(2022,11,20).timestamp()*1000))):
    url = 'https://api.binance.com/api/v3/klines'
    interval = '1d'
   

    par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end}

    data = pd.DataFrame(json.loads(requests.get(url, params= par).text))

    #format columns name

    data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']

    data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]

    data=data.astype(float)
    return data

In [ ]:
BTC_prices1 = pd.DataFrame(get_data("BTCUSDT", start = str(int(dt.datetime(2020,4,17).timestamp()*1000)),  
             end = str(int(dt.datetime(2021,8,12).timestamp()*1000)))['close'])
BTC_prices1.index.name = 'date'
BTC_prices2 = pd.DataFrame(get_data("BTCUSDT", start = str(int(dt.datetime(2021,8,12).timestamp()*1000)),  
             end = str(int(dt.datetime(2022,11,19).timestamp()*1000)))['close'])
BTC_prices2.index.name = 'date'
BTC_prices = pd.concat([BTC_prices1, BTC_prices2])


ETH_prices1 = pd.DataFrame(get_data("ETHUSDT", start = str(int(dt.datetime(2020,4,17).timestamp()*1000)),  
             end = str(int(dt.datetime(2021,8,12).timestamp()*1000)))['close'])
ETH_prices1.index.name = 'date'
ETH_prices2 = pd.DataFrame(get_data("ETHUSDT", start = str(int(dt.datetime(2021,8,12).timestamp()*1000)),  
             end = str(int(dt.datetime(2022,11,19).timestamp()*1000)))['close'])
ETH_prices2.index.name = 'date'
ETH_prices = pd.concat([ETH_prices1, ETH_prices2])
ETH_prices

merged_df = pd.merge(ETH_prices, BTC_prices, on='date', how='inner')

merged_df = merged_df.rename(columns={'close_x':'ETH','close_y': 'BTC' })
merged_df['Date'] = merged_df.index

# Reset index and create a new index
merged_df = merged_df.reset_index(drop=True)
merged_df['date'] = merged_df['Date'].dt.date
merged_df = merged_df.drop('Date', axis=1)
merged_df

In [ ]:
d = {'date':merged_df.date , "Price":np.divide(merged_df.ETH.values,merged_df.BTC.values)}
target_price = pd.DataFrame(data = d)
target_price['log_returns'] = np.log(target_price.Price/target_price.Price.shift(1))
target_price.dropna(inplace = True)


# Calculate the rolling standard deviation using a 30-day window
rolling_volatility = target_price['log_returns'].rolling(window=30).std()

# Convert the rolling volatility to percentage format if desired
rolling_volatility_percentage = rolling_volatility * 100 
target_price['rolling_volatility_percentage'] = rolling_volatility_percentage
target_price = target_price.fillna(method='backfill')
target_price.reset_index(inplace = True)
target_price

In [ ]:
BTC_ETH

In [ ]:
BTC_ETH_grouped = BTC_ETH.groupby(['date', 'Event']).sum()
BTC_ETH_grouped

In [ ]:
#BTC_ETH_grouped['date'] = pd.to_datetime(BTC_ETH_grouped['date'])
merged_df.set_index('date', inplace=True)
# Perform the merge
BTC_ETH = BTC_ETH_grouped.join(merged_df, how='left')

In [ ]:
BTC_ETH

In [ ]:
def prepare_df(df, digits_base, digits_sec):
    df['amount0In'], df['amount0Out'] = df['amount0In']/digits_base, df['amount0Out']/digits_base
    df['amount1In'], df['amount1Out'] = df['amount1In']/digits_sec, df['amount1Out']/digits_sec
    in0_sum = np.array([])
    in1_sum = np.array([])
    out0_sum = np.array([])
    out1_sum = np.array([])
    
    for i in range(0,len(df)):
        
        in0 = df[["amount0In", "amount1In", "amount0Out", "amount1Out"]].loc[:i].sum()['amount0In']
        in0_sum = np.append(in0_sum, in0)
        out0 = df[["amount0In", "amount1In", "amount0Out", "amount1Out"]].loc[:i].sum()['amount0Out']
        out0_sum = np.append(out0_sum, out0)
        in1 = df[["amount0In", "amount1In", "amount0Out", "amount1Out"]].loc[:i].sum()['amount1In']
        in1_sum = np.append(in1_sum, in1)
        out1 = df[["amount0In", "amount1In", "amount0Out", "amount1Out"]].loc[:i].sum()['amount1Out']
        out1_sum = np.append(out1_sum, out1)
        
    df['in0_sum'] = in0_sum
    df['in1_sum'] = in1_sum
    df['out0_sum'] = out0_sum
    df['out1_sum'] = out1_sum
    
    return df

def iloss(ratio, numerical=True):
    il = 2*np.sqrt(ratio)/(ratio+1)-1
    r = f"{il:.2%}" if not numerical else il
    return r

In [ ]:
BTC_ETH_1 = prepare_df(BTC_ETH.reset_index(), digits_base=10**8, digits_sec=10**18)

In [ ]:
BTC_ETH_grouped = BTC_ETH_1.groupby(['date', 'Event']).sum()
BTC_ETH_grouped

# open hedge, when enter the position, close it only when close the position 

In [ ]:
def perfect_providing(df, daily_hedge):
    my_usd = 200
    signal = []
    P_0 = df.loc[('2020-05-18', 'Swap')]['ETH']/df.loc[('2020-05-18', 'Swap')]['BTC']
    my_fees, PnL = np.array([]), np.array([])
    for group, group_df in df.iterrows():
        
        if 'Swap' == group[1]:
            
            P_1 = group_df['ETH']/ group_df['BTC']
            fees = (group_df['amount0In']*group_df['BTC'] + group_df['amount1In']*group_df['ETH'])*0.003
            
            pool_0_usd = (group_df['in0_sum']- group_df['out0_sum'])*group_df['BTC']
            pool_1_usd = (group_df['in1_sum']- group_df['out1_sum'])*group_df['ETH']
            
            total_pool_usd = pool_0_usd+pool_1_usd
            
            alpha = fees/total_pool_usd
            il = iloss(P_1/P_0)
            
            if 100*(alpha+il)>0:
                signal.append(True)
                my_fees = np.append(my_fees, fees*my_usd/total_pool_usd)
                my_usd += fees*my_usd/total_pool_usd 
                #print('alpha-il',alpha-il)
                if daily_hedge:
                    PnL = np.append(PnL,alpha+il)
                    P_0 = P_1
                    
                else:
                    if group_df['shifted_signal']:
                        PnL = np.append(PnL,alpha+0)
                        
                    else:
                        PnL = np.append(PnL,alpha+il)
                
            else:
                signal.append(False)
                my_fees = np.append(my_fees, 0) 
                PnL = np.append(PnL,0)
                P_0 = P_1

    return my_fees , PnL, signal

In [ ]:
my_fees , PnL, signal = perfect_providing(BTC_ETH_grouped, daily_hedge=False)

In [ ]:
my_fees_hedge , PnL_hedge, signal_hedge = perfect_providing(BTC_ETH_grouped, daily_hedge=True)

In [ ]:
5.753905e-08

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set the figure size
fig, axs = plt.subplots(2, 1, figsize=(20, 20), sharex=True)

times = merged_df.date[31:]

# Calculate the cumulative PnL
cumulative_fees_with_hedge = np.cumsum(my_fees_hedge)

# Plot the cumulative PnL with different colors for zero values
axs[0].scatter(times[signal_hedge], cumulative_fees_with_hedge[signal_hedge], color='blue', s=1, label='Providing')
axs[0].scatter(times[ np.logical_not(signal_hedge)], cumulative_fees_with_hedge[ np.logical_not(signal_hedge)], color='red', s=10, label='Not Providing')

axs[0].set_title(label='Perfect liquidity providing with daily hedge')
axs[0].set_ylabel('cumulative_fees_with_hedge')
axs[0].grid(True)

# Plot the BTC/ETH normalized values
cumulative_fees = np.cumsum(my_fees)

# Plot the cumulative PnL with different colors for zero values
axs[1].scatter(times[signal ], cumulative_fees[signal ], color='blue', s=1)
axs[1].scatter(times[ np.logical_not(signal) ], cumulative_fees[ np.logical_not(signal)], color='red', s=10)

axs[1].set_ylabel('cumulative_fees')
axs[1].grid(True)

fig.autofmt_xdate(which='both')

# Save the figure with higher DPI
fig.savefig("cumulative_fees_Perfect_liq_daily_hedge.png", dpi=300)

# Show the figure
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set the figure size
fig, axs = plt.subplots(3, 1, figsize=(20, 25), sharex=True)

times = merged_df.date[31:]

# Calculate the cumulative PnL
cumulative_pnl_hedge = (np.cumprod(PnL_hedge + 1) - 1) * 100

# Create boolean masks for equal and not equal adjacent values in PnL
mask_equal = np.equal(PnL_hedge[:-1], PnL_hedge[1:])
mask_not_equal = np.logical_not(mask_equal)

# Plot the cumulative PnL with different colors for equal and not equal adjacent values
axs[0].plot(times[1:], cumulative_pnl_hedge[1:], color='blue')
axs[0].scatter(times[:-1][mask_not_equal], cumulative_pnl_hedge[:-1][mask_not_equal], color='blue', s=1, label='Providing')
axs[0].scatter(times[my_fees_hedge > 0], cumulative_pnl_hedge[my_fees_hedge > 0], color='red', s=5, label='Not Providing')

axs[0].set_title(label='PnL comparison under perfect liquidity providing')
axs[0].set_ylabel('PnL_hedge in %')
axs[0].grid(True)
axs[0].legend()  # Add legend

cumulative_pnl = (np.cumprod(PnL + 1) - 1) * 100

mask_equal = np.equal(PnL[:-1], PnL[1:])
mask_not_equal = np.logical_not(mask_equal)


# Plot the cumulative PnL with different colors for equal and not equal adjacent values
axs[1].plot(times[1:], cumulative_pnl[1:], color='blue')
axs[1].scatter(times[:-1][mask_not_equal], cumulative_pnl[:-1][mask_not_equal], color='blue', s=1, label='Providing')
axs[1].scatter(times[my_fees > 0], cumulative_pnl[my_fees > 0], color='red', s=5, label='Not Providing')

axs[1].set_ylabel('PnL in %')
axs[1].grid(True)

axs[2].plot(times, (cumulative_pnl_hedge - cumulative_pnl))
axs[2].set_ylabel('PnL_hedge - PnL in p.p.')
axs[2].grid(True)

fig.autofmt_xdate(which='major')

# Spacing out the x-axis tick labels
num_ticks = 10  # Number of desired tick labels
x_ticks = np.linspace(31, len(times) - 2, num_ticks, dtype=int)
x_tick_labels = times[:-1][x_ticks]

axs[2].set_xticks(times[:-1][x_ticks])  # Set x-axis tick locations
axs[2].set_xticklabels(x_tick_labels, rotation=45, ha='right')  # Set x-axis tick labels with rotation

# Save the figure with higher DPI
fig.savefig("PnL_comp_Perfect_liq_daily_hedge.png", dpi=300)

# Show the figure
plt.show()

In [ ]:
bool_ = my_fees > 0
num_true = np.count_nonzero(bool_)
print(num_true/len(bool_))

In [ ]:
def calculate_month_alpha(df):
    alphas_month = []
    for group, group_df in df.iterrows():
        
        if 'Swap' == group[1]:
            
            fees = (group_df['amount0In']*group_df['BTC'] + group_df['amount1In']*group_df['ETH'])*0.003
            
            pool_0_usd = (group_df['in0_sum']- group_df['out0_sum'])*group_df['BTC']
            pool_1_usd = (group_df['in1_sum']- group_df['out1_sum'])*group_df['ETH']
            
            total_pool_usd = pool_0_usd+pool_1_usd
            
            alpha_day = fees/total_pool_usd
            
            alpha_month = (1+alpha_day)**365-1
            
            alphas_month.append(alpha_month)
            
    return alphas_month

# $\mu = \frac{1}{30}\sum_{i=1}^{30} \log\left(\frac{P_t}{P_{t-1}}\right) + \frac{\sigma^2}{2}$

In [ ]:
alphas_month = calculate_month_alpha(BTC_ETH_grouped)

In [ ]:
alphas_month

In [ ]:
d = {'date':merged_df['date'] , "Price":np.divide(merged_df.ETH.values,merged_df.BTC.values)}
target_price = pd.DataFrame(data = d)
target_price['log_returns'] = np.log(target_price.Price/target_price.Price.shift(1))
target_price.dropna(inplace = True)


# Calculate the rolling standard deviation using a 30-day window
rolling_volatility = target_price['log_returns'].rolling(window=30).std()
rolling_ret = target_price['log_returns'].rolling(window=30).mean()

target_price['rolling_volatility'] = rolling_volatility* np.sqrt(365)
target_price['rolling_ret'] = rolling_ret
target_price = target_price.dropna()
#target_price = target_price.drop('index', axis=1)
target_price = target_price.drop(target_price.index[0])
target_price.reset_index(inplace = True)
target_price

In [ ]:
target_price['alpha'] = alphas_month
target_price = target_price.drop('index', axis=1)
target_price

In [ ]:
def calculate_IV(df):
    IV = []
    for i in range(len(df)):
        day = df.loc[i]
        t = 30/365
        mu = day['rolling_ret'] + day['rolling_volatility']**2/2
        alpha = day['alpha']
        ln = np.log(np.cosh(mu*t/2))
        iv = np.sqrt(8/t*(alpha*t-ln))
        print(iv)

        IV.append(iv)
        
    return IV

In [ ]:
IV = calculate_IV(target_price)

In [ ]:
target_price["IV"] = IV

In [ ]:
plt.plot(IV, label='IV')
plt.plot(target_price['rolling_volatility'])

In [ ]:
result = np.greater(IV, target_price['rolling_volatility'].values)
np.count_nonzero(result)

In [ ]:
target_price['signal'] = result

In [ ]:
target_price['shifted_signal'] = target_price['signal'].shift(-1)

In [ ]:
target_price

In [ ]:
#BTC_ETH_grouped = BTC_ETH_grouped.join(target_price['signal'])
BTC_ETH_grouped = BTC_ETH_grouped.join(target_price['shifted_signal'])

In [ ]:
BTC_ETH_grouped = BTC_ETH_grouped.drop('shifted_signal', axis=1)
BTC_ETH_grouped = BTC_ETH_grouped.join(target_price['shifted_signal'])
BTC_ETH_grouped

In [ ]:
def vol_strategy_providing(df, daily_hedge):
    my_usd = 200
    P_0 = df.loc[('2020-05-18', 'Swap')]['ETH']/df.loc[('2020-05-18', 'Swap')]['BTC']
    my_fees, PnL = np.array([0]), np.array([0])
    for group, group_df in df.iterrows():
        
        if 'Swap' == group[1]:
            
            P_1 = group_df['ETH']/ group_df['BTC']
            fees = (group_df['amount0In']*group_df['BTC'] + group_df['amount1In']*group_df['ETH'])*0.003
            
            pool_0_usd = (group_df['in0_sum']- group_df['out0_sum'])*group_df['BTC']
            pool_1_usd = (group_df['in1_sum']- group_df['out1_sum'])*group_df['ETH']
            
            total_pool_usd = pool_0_usd+pool_1_usd
            
            alpha = fees/total_pool_usd
            #print('alpha:', round(100*alpha,2))
            il = iloss(P_1/P_0)
            
           
            if group_df['signal']:
                my_fees = np.append(my_fees, fees*my_usd/total_pool_usd)
                my_usd += fees*my_usd/total_pool_usd 
                #print('alpha-il',alpha-il)
                if daily_hedge:
                    PnL = np.append(PnL,alpha+il)
                    P_0 = P_1
                    
                else:
                    if group_df['shifted_signal']:
                        PnL = np.append(PnL,alpha+0)
                        
                    else:
                        PnL = np.append(PnL,alpha+il)
                    
            else:
                my_fees = np.append(my_fees, 0) 
                PnL = np.append(PnL,0)
                P_0 = P_1

    return my_fees , PnL

In [ ]:
my_fees , PnL = vol_strategy_providing(BTC_ETH_grouped,daily_hedge=0)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set the figure size
fig, axs = plt.subplots(3, 1, figsize=(20, 30), sharex=True)

times = merged_df.date[30:]

# Calculate the cumulative PnL
cumulative_pnl = (np.cumprod(PnL+1)-1)*100

# Create boolean masks for equal and not equal adjacent values in PnL
mask_equal = result
mask_not_equal = np.logical_not(mask_equal)

# Plot the cumulative PnL with different colors for equal and not equal adjacent values
axs[0].plot(times[1:], cumulative_pnl[1:], color='blue')
axs[0].scatter(times[:-1][mask_equal], cumulative_pnl[:-1][mask_equal], color='red', s=10, label='Not Providing Liquidity')
axs[0].scatter(times[:-1][mask_not_equal], cumulative_pnl[:-1][mask_not_equal], color='blue', s=1, label='Providing')

axs[0].set_title(label='Strategy liquidity providing')
axs[0].set_ylabel('PnL in %')
axs[0].grid(True)
axs[0].legend()  # Add legend

# Plot the BTC/ETH normalized values
axs[1].plot(times[1:], IV, color='green', label='IV')
axs[1].plot(times[1:], target_price.rolling_volatility.values, color='red', label='HV')
axs[1].set_ylabel('Volatility')
axs[1].set_xlabel('Date')
axs[1].grid(True)
# Plot the BTC/ETH normalized values
axs[2].plot(times, (merged_df['BTC'][30:] / merged_df['BTC'][30]) / (merged_df['ETH'][30:] / merged_df['ETH'][30]), color='gray')

axs[2].set_ylabel('ETH/BTC NORMALIZED')
axs[2].grid(True)

fig.autofmt_xdate(which='both')

# Save the figure with higher DPI
fig.savefig("PnL_strategy_liq.png", dpi=300)

# Show the figure
plt.show()


In [ ]:
my_fees_h , PnL_h = vol_strategy_providing(BTC_ETH_grouped,daily_hedge=1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set the figure size
fig, axs = plt.subplots(3, 1, figsize=(20, 30), sharex=True)

times = merged_df.date[30:]

# Calculate the cumulative PnL
cumulative_pnl = (np.cumprod(PnL_h+1)-1)*100

# Create boolean masks for equal and not equal adjacent values in PnL
mask_equal = result
mask_not_equal = np.logical_not(mask_equal)

# Plot the cumulative PnL with different colors for equal and not equal adjacent values
axs[0].plot(times[1:], cumulative_pnl[1:], color='blue')
axs[0].scatter(times[:-1][mask_equal], cumulative_pnl[:-1][mask_equal], color='red', s=10, label='Not Providing')
axs[0].scatter(times[:-1][mask_not_equal], cumulative_pnl[:-1][mask_not_equal], color='blue', s=1, label='Providing')

axs[0].set_title(label='Strategy liquidity providing')
axs[0].set_ylabel('PnL in % with hedge')
axs[0].grid(True)
axs[0].legend()  # Add legend

# Plot the BTC/ETH normalized values
axs[1].plot(times[1:], IV, color='green', label='IV')
axs[1].plot(times[1:], target_price.rolling_volatility.values, color='red', label='HV')
axs[1].set_ylabel('Volatility')
axs[1].set_xlabel('Date')
axs[1].grid(True)
# Plot the BTC/ETH normalized values
axs[2].plot(times, (merged_df['BTC'][30:] / merged_df['BTC'][30]) / (merged_df['ETH'][30:] / merged_df['ETH'][30]), color='gray')

axs[2].set_ylabel('ETH/BTC NORMALIZED')
axs[2].grid(True)

fig.autofmt_xdate(which='both')

# Save the figure with higher DPI
fig.savefig("PnL_strategy_liq_daily_hedge.png", dpi=300)

# Show the figure
plt.show()
